TODO:


1. get new datasets
    - UCI (NodeFlow and kiran paper) both has target scaling of [-1,1] for all input features and target
2. implement kiran mlp resnet
3. run it on datassts from papaers (nodeflow) or any paper that is good 
4. check if results match with the nodeflow paper , or openML-ctr-23

- torch nn.embeddings for cateogical data later on.

- look at kiran resnet/mlp for univariate regression then:

Obseevations:
- tabpfn uses ctr23 and AutoML, what a suprrise!

# Code Test

In [2]:
import openml


dataset = openml.datasets.get_dataset(44962)
X, y, categorical_indicator, attribute_names = dataset.get_data(target=dataset.default_target_attribute)  

# Manual fix for forest_fires dataset (ID 44962) with day and month
if True:
    categorical_indicator[2] = True  # month
    categorical_indicator[3] = True  # day


categorical_cols = [col for col, is_cat in zip(X.columns, categorical_indicator) if is_cat]
numerical_cols = [col for col in X.columns if col not in categorical_cols]

print(attribute_names)
print(categorical_indicator)
print(categorical_cols)

['X', 'Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']
[False, False, True, True, False, False, False, False, False, False, False, False]
['month', 'day']


In [8]:
from configs.config import DATASETS
from utils.data_loader import load_preprocessed_data
import numpy as np

'''
TODO:

1. get new datasets
    - UCI (NodeFlow and kiran paper) both has target scaling of [-1,1] for all input features and target
    - multivariate ones are downloaded from https://github.com/gpapamak/maf/tree/master?tab=readme-ov-file#how-to-get-the-datasets
      they are already in procssed format and pre-processing is taken also from there
        - hepmass has no regression, mode details in  todo, for now adapted naval as multi
        - miniboone also has no regression, in todo
    - OpenML all done, auto encoder as per the paper they used.

2. implement kiran mlp resnet

3. run it on datassts from papaers (nodeflow) or any paper that is good 
4. check if results match with the nodeflow paper , or openML-ctr-23

- torch nn.embeddings for cateogical data later on.

- look at kiran resnet/mlp for univariate regression then:

Obseevations:
- tabpfn uses ctr23 and AutoML, what a suprrise!

'''
if __name__ == '__main__':
    source = "openml_ctr23"
    print_info = False

    for dataset_key, dataset_info in DATASETS[source].items():
        X_train, X_test, X_val, y_train, y_test, y_val, dataset_name = load_preprocessed_data(source, dataset_key)

        # Identify categorical or non-numeric columns
        numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
        
        # Infer categorical columns (object or category dtypes)
        # This assumes that if a column is not numeric, it's categorical.
        # Or, if you have explicit categorical_indicator from openml, use that.
        potential_cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
        
        if print_info:
            print(f"Testing dataset: {dataset_key} ({dataset_info['name']})")
            print(f"  X_train shape: {X_train.shape}")
            print(f"  X_test shape: {X_test.shape}")
            print(f"  X_val shape: {X_test.shape}")
            print(f"  y_train shape: {y_train.shape}")
            print(f"  y_test shape: {y_test.shape}")
            print(f"  y_val shape: {y_val.shape}\n")

        print(f"numerical columns in {dataset_name}: {numerical_cols}")
        print(f"Categorical columns in {dataset_name}: {potential_cat_cols}\n")


17:36:00 - INFO: fetching auction_verification (44958) from openML.
17:36:00 - INFO: One-hot encoding categorical features for dataset auction_verification: ['property.product', 'property.winner']
17:36:00 - INFO: fetching airfoil_self_noise (44957) from openML.
17:36:00 - INFO: fetching brazilian_houses (44990) from openML.
17:36:00 - INFO: One-hot encoding categorical features for dataset brazilian_houses: ['city', 'floor', 'animal', 'furniture']
17:36:00 - INFO: fetching cars (44994) from openML.
17:36:00 - INFO: fetching forest_fires (44962) from openML.
17:36:00 - INFO: fetching QSAR_fish_toxicity (44970) from openML.
17:36:01 - INFO: fetching space_ga (45402) from openML.
17:36:01 - INFO: fetching socmob (44987) from openML.
17:36:01 - INFO: One-hot encoding categorical features for dataset socmob: ['fathers_occupation', 'sons_occupation', 'family_structure', 'race']
17:36:01 - INFO: fetching geographical_origin_of_music (44965) from openML.


numerical columns in auction_verification: ['process.b1.capacity', 'process.b2.capacity', 'process.b3.capacity', 'process.b4.capacity', 'property.price']
Categorical columns in auction_verification: []

numerical columns in airfoil_self_noise: ['frequency', 'angle_of_attack', 'chord_length', 'free_stream_velocity', 'displacement_thickness']
Categorical columns in airfoil_self_noise: []

numerical columns in brazilian_houses: ['area', 'rooms', 'bathroom', 'parking_spaces', 'hoa']
Categorical columns in brazilian_houses: []

numerical columns in cars: ['Mileage', 'Cylinder', 'Doors', 'Cruise', 'Sound', 'Leather', 'Buick', 'Cadillac', 'Chevy', 'Pontiac', 'Saab', 'Saturn', 'convertible', 'coupe', 'hatchback', 'sedan', 'wagon']
Categorical columns in cars: []

numerical columns in forest_fires: ['X', 'Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']
Categorical columns in forest_fires: ['month', 'day']

numerical columns in QSAR_fish_toxicity: ['CIC0', 'SM1_Dz', 'GATS1i', 'NdsC

In [1]:
from configs.config import DATASETS
from utils.data_loader import load_preprocessed_data

source = "uci"
# For looping through all datasets in the source
datasets_to_run = DATASETS.get(source, {})


for dataset_key, dataset_info_dict in datasets_to_run.items():
    dataset_display_name = dataset_info_dict.get('name', dataset_key)
    #print(f"\n--- Processing dataset: {dataset_key} ({dataset_display_name}) ---")

    train_loader, val_loader, test_loader, num_numerical_features, dataset_name = \
                load_preprocessed_data(source, dataset_key, batch_size=2048)
    
    if True:
        print(f"Dataset: {dataset_name}")
        print(f"Number of numerical features: {num_numerical_features}")
        print(f"Train loader batches: {len(train_loader)}, Val loader batches: {len(val_loader)}, Test loader batches: {len(test_loader)}")


20:43:40 - INFO: fetching Concrete Compressive Strength (concrete) from UCI.
20:43:41 - INFO: Processing Concrete Compressive Strength...
20:43:41 - INFO: fetching Energy Efficiency (energy) from UCI.
20:43:41 - INFO: Processing Energy Efficiency...
20:43:41 - INFO: fetching Condition Based Maintenance of Naval Propulsion Plants (naval) from uci as zip.


concrete dataset, N=1030, d=8
Dataset: Concrete Compressive Strength
Number of numerical features: 10
Train loader batches: 1, Val loader batches: 1, Test loader batches: 1
energy dataset, N=768, d=8
Dataset: Energy Efficiency
Number of numerical features: 10
Train loader batches: 1, Val loader batches: 1, Test loader batches: 1


20:43:41 - INFO: Processing Condition Based Maintenance of Naval Propulsion Plants...
20:43:41 - INFO: fetching Combined Cycle Power Plant (power) from ucirepo.


Dataset: Condition Based Maintenance of Naval Propulsion Plants
Number of numerical features: 17
Train loader batches: 5, Val loader batches: 2, Test loader batches: 1


20:43:44 - INFO: Processing Combined Cycle Power Plant...
20:43:44 - INFO: fetching Physicochemical Properties of Protein Tertiary Structure (protein) from UCI.
20:43:45 - INFO: Processing Physicochemical Properties of Protein Tertiary Structure...


Dataset: Combined Cycle Power Plant
Number of numerical features: 4
Train loader batches: 4, Val loader batches: 1, Test loader batches: 1
protein dataset, N=45730, d=9


20:43:45 - INFO: fetching Wine Quality (wine) from UCI.
20:43:45 - INFO: Processing Wine Quality...
20:43:45 - INFO: fetching Yacht Hydrodynamics (yacht) from UCI.
20:43:45 - INFO: Processing Yacht Hydrodynamics...
20:43:45 - INFO: fetching Kinematics 8nm, ID: 189 from OpenML.
20:43:45 - INFO: Processing Kinematics 8nm...


Dataset: Physicochemical Properties of Protein Tertiary Structure
Number of numerical features: 11
Train loader batches: 17, Val loader batches: 5, Test loader batches: 3
wine dataset, N=1599, d=11
Dataset: Wine Quality
Number of numerical features: 13
Train loader batches: 1, Val loader batches: 1, Test loader batches: 1
yacht dataset, N=308, d=6
Dataset: Yacht Hydrodynamics
Number of numerical features: 8
Train loader batches: 1, Val loader batches: 1, Test loader batches: 1
Dataset: Kinematics 8nm
Number of numerical features: 8
Train loader batches: 3, Val loader batches: 1, Test loader batches: 1
